In [2]:
from pyspark.sql import SparkSession, functions as F, types as T

In [3]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [4]:
print(spark.version)

3.5.0


In [7]:
#spark.stop()

In [5]:
#actor_df = spark.read.csv('./data/actor.csv', header=True, inferSchema=True)


In [5]:
actor_df = spark.read.csv('./data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('./data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('./data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('./data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('./data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('./data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('./data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('./data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('./data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('./data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('./data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('./data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('./data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('./data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [28]:
# тут має бути розвʼязок задачі

film_df.join(film_category_df, on = 'film_id').join(category_df, on='category_id')\
    .select('category_id','name').groupBy('category_id','name').count().orderBy('count', ascending=False).show(5)


+-----------+-----------+-----+
|category_id|       name|count|
+-----------+-----------+-----+
|         15|     Sports|   74|
|          9|    Foreign|   73|
|          8|     Family|   69|
|          6|Documentary|   68|
|          2|  Animation|   66|
+-----------+-----------+-----+
only showing top 5 rows



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [29]:
# тут має бути розвʼязок задачі
from pyspark.sql.functions import  desc, max
actor_df.join(film_actor_df, on = 'actor_id').join(film_df, on = 'film_id').groupBy('rental_duration','last_name', 'first_name') \
    .agg(max("rental_duration").alias("max_rental_duration")) \
    .select('rental_duration','last_name', 'first_name') \
    .sort(desc("rental_duration")) \
    .show(10)


+---------------+---------+----------+
|rental_duration|last_name|first_name|
+---------------+---------+----------+
|              7|  WINSLET|       FAY|
|              7| WILLIAMS|    MORGAN|
|              7|     DERN| SYLVESTER|
|              7|  HOPKINS|   NATALIE|
|              7|    GRANT|      ADAM|
|              7|DEGENERES|      GINA|
|              7|    ALLEN|      CUBA|
|              7|    DENCH|  JULIANNE|
|              7|   MALDEN|     GRETA|
|              7| JOVOVICH|      KIRK|
+---------------+---------+----------+
only showing top 10 rows



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [45]:
# тут має бути розвʼязок задачі
from pyspark.sql.functions import  col, desc, max

max_value_df = film_df.join(payment_df, film_df.film_id == payment_df.rental_id) \
    .select(max(col('rental_duration')*col('amount')).alias('max_sum'))

film_df.join(film_category_df, on = 'film_id').join(category_df, on='category_id').join(payment_df, film_df.film_id == payment_df.rental_id) \
    .join(max_value_df, film_df.rental_duration*payment_df.amount >= max_value_df.max_sum ) \
    .select(category_df.name) \
    .distinct().show(10)


+---------+
|     name|
+---------+
|   Travel|
|Animation|
|   Horror|
|      New|
| Children|
+---------+



4.
Вивести назви фільмів, яких не має в inventory.

In [10]:
# тут має бути розвʼязок задачі

df = film_df.join(inventory_df, film_df.film_id == inventory_df.film_id, "left") \
    .filter(inventory_df.film_id.isNull()) \
    .select(film_df.film_id,film_df.title).distinct().show(10)

+-------+--------------------+
|film_id|               title|
+-------+--------------------+
|    318|   FIREHOUSE VIETNAM|
|    144| CHINATOWN GLADIATOR|
|    642|      ORDER BETRAYED|
|    607|         MUPPET MILE|
|    802|         SKY MIRACLE|
|    712|   RAIDERS ANTITRUST|
|    128|       CATCH AMISTAD|
|     41|ARSENIC INDEPENDENCE|
|    943|   VILLAIN DESPERATE|
|     87|   BOONDOCK BALLROOM|
+-------+--------------------+
only showing top 10 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [27]:
# тут має бути розвʼязок задачі
from pyspark.sql import Window, functions as F
from pyspark.sql.functions import  desc
w = Window.partitionBy(actor_df.actor_id)

actor_df.join(film_actor_df, on = 'actor_id').join(film_df, film_actor_df.film_id == film_df.film_id) \
    .join(film_category_df, film_category_df.film_id == film_df.film_id) \
    .join(category_df, category_df.category_id == film_category_df.category_id) \
    .filter(category_df.name == 'Children') \
    .select(actor_df.last_name, actor_df.first_name, \
            #film_df.film_id, film_df.title, \
            F.count(category_df.category_id).over(w).alias('cnt_actor')) \
    .distinct() \
    .sort(desc('cnt_actor')) \
    .show(3)

    


    

+---------+----------+---------+
|last_name|first_name|cnt_actor|
+---------+----------+---------+
|   VOIGHT|     HELEN|        7|
|  GARLAND|     KEVIN|        5|
|    TANDY|      MARY|        5|
+---------+----------+---------+
only showing top 3 rows



Stop Spark session:

In [30]:
spark.stop()